This notebook is a step by step guide about how to train a deep neural network (DNN) in the DeepDeconv framework.

In [1]:
## Set up the sys.path in order to be able to import our modules
import os
import sys
module_path = os.path.abspath(os.path.join('../python'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k

###################################
# TensorFlow wizardry
config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
# This line is optional, don't add it unless you really need to set a limit on the memory available for your process
# For instance, if you want to train 2 DNNs on the same GPU without one overlapping the memory needed by the other
# Change the value to set the percentage of memory allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.75 

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Now you can create/load your DNN

Using TensorFlow backend.


We load the class of the network. This class must inherit from the DeepNet superclass. The method build_model has to be redefined in the child class with the wanted architecture. In our work, we use the network defined in deconvnNet. 

In [3]:
from DeepDeconv.deepnetFCS.DeconvNet_custom import DeconvNet

In [4]:
nb_scales = 2 #4
growth_rate = 4 #12
nb_layers_per_block = [2,2]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.1
shape_constraint=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shape_constraint=shape_constraint)



/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeconvNet_custom.py:293: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=[<tf.Tenso...)`
  self.model = Model(input = [inputs,window,norm], outputs =output)


Tensor("loss/conv2d_4_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss/conv2d_4_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 96, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 96, 32)   128         conv2d_1[0][0]                   
___________________________________________________

So now that we have our network ready, the next step is to train it.

During my internship, I used .fits files containing 10 000 galaxies each of size (96x96). Each file contains 4 HDU:
HDU 0: noisy convolved galaxies (never used those)
HDU 1: noiseless convolved galaxies
HDU 2: target galaxies convolved with gaussian PSF of FWHM=0.15 (irrelevent)
HDU 3: Euclid PSFs
HDU 4 target galaxies convolved with gaussian PSF of FWHM=0.07
Each HDU is one big image 9600x9600 containing the mosaic of the 10 000 samples

The DeepNet class has methods to train the network. 3 alternatives are possible:

-The data is already loaded in python, use:

    train(train_data, model_file = '', epochs=20, batch_size=32, validation_split=0.1)
train_data is a numpy array of shape (2, nb_samples, img_row, img_col, 1), the first dimension containing all the inputs of the network and the second dimension the ground truths. validation_split set the percentage of data for the valisation set. This method load your whole dataset directly so if you work with a huge set, you will pobably run out of memory, especially on GPU. You should try to opt for the other methods if possible.
    
-The data is already preprocessed inside .npy files and you want to load it one batch at a time:

    train_generator_npy(train_files, validation_file, epochs=20, batch_size=32, nb_img_per_file=10000)
as above, each file must contain an array of shape (2, nb_img_per_file, img_row, img_col, 1). A validation_file containing the validation set must be provided too.

-You want to read the fits and process the data on the fly:

    train_generator(train_files, validation_file, epochs=20, batch_size=32,
                        nb_img_per_file=10000, validation_set_size=10000,
                        noise_std=None, SNR=None,
                        noiseless_obs_hdu=0, targets_hdu=0, psf_hdu=0,
                        image_dim=96, image_per_row=100,
                        deconv_mode=None, rho_fista=1e-3)
so here you provides the list of fits files inside train_files and one validation file inside validation_files. You use hdu parameters to extract the desired values inside the fits. Functions inside batch_utils.py will automatically process the data from the fits and convert it to numpy array of the correct shape for the network.

In our case, we use this function as follows:

train_files (list of str): list of all our fits files containing the training set.

validation_file (str): validation set inside a fits file.

noise_std (int or list of 2 ints or None): int for constant noise, list of 2 ints [min_std, max_std] for a range of sigma and None if you want to use SNR instead.

SNR (int or list of 2 ints or None): same as noise_std but you input SNR values instead.

noiseless_obs_hdu (int): HDU in which the noiseless convolved galaxies are. Noise will be added according to the previous parameters.

targets_hdu (int): HDU containing the ground truths.

psf_hdu (int): HDU containing the PSFs.

image_dim (int): nb of rows/columns of each galaxy (we use square images).

image_per_row (int): images per row inside the fits mosaic (we use square mosaic in our fits files).

deconv_mode (None or 'TIKHONOV' or 'FISTA'): set the preprocessing to be applied on the input of the network.

rho_fista (float): only needed as a parameter for FISTA pre-processing.


The 2nd method is the best if you have limited computational ressources (but you have to do the preprocessing beforehand to generate all the .npy files).
The 3rd method provides an efficient method to do the pre-processing of each batch right before feeding it to the network for training. We use this method as an example in the following cells.
Both use the fit_generator method inside the model class in Keras. This function is the best to train the network without overloading your memory. It requires to define a generator, see dynamic_batches in batch_utils.npy for an example of implementation.

Note that for all the methods, a callback save automatically the network which got the best result on validation data (using MSE). If model_file is left empty, the model will be saved in the current working directory as "network_name.hdf5". Input the string containing the wanted path/name in model_file (with .hdf5 extension) to chose save location.

In [ ]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'image-*-multihdu.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(write_path+'window/'+'Gaussian*')
win_files.sort()
print(win_files)


SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],
                        win_filename=win_files[2:],win_hdu=0,mom_hdu=1)

['/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-0-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-1-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-10-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-11-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-12-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-13-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-14-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-15-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-16-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-17-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-18-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-19-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-2-mult

/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:195: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=20, validation_data=([array([[..., verbose=1, callbacks=[<keras.ca..., initial_epoch=0, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


Epoch 1/20
5325/5947 [=========================>....] - ETA: 5:31 - loss: 120.7836 - mean_squared_error: 0.2657 - shape_metric: 120.5179

In [ ]:
del dnn

In [ ]:
nb_scales = 2 #4
growth_rate = 4 #12
nb_layers_per_block = [2,2]#[4,5,6,7]
activation_function= 'relu' #'relu'

atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate)
if resNet:
    network_name+='_resNet'

dnn= DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou)

In [ ]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'image-*-multihdu.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(write_path+'window/'+'Gaussian*')
win_files.sort()
print(win_files)


SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],
                        win_filename=win_files[2:],win_hdu=0,mom_hdu=1)

In [ ]:
del dnn

In [4]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.1
shape_constraint=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}'.format(nb_scales,layer_string,
                                                                                        activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn= DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shape_constraint=shape_constraint)

/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeconvNet_custom.py:293: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=[<tf.Tenso...)`
  self.model = Model(input = [inputs,window,norm], outputs =output)


Tensor("loss/conv2d_8_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss/conv2d_8_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 96, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 96, 32)   128         conv2d_1[0][0]                   
___________________________________________________

In [ ]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'image-*-multihdu.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(write_path+'window/'+'Gaussian*')
win_files.sort()
print(win_files)


SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],
                        win_filename=win_files[2:],win_hdu=0,mom_hdu=1)

['/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-0-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-1-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-10-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-11-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-12-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-13-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-14-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-15-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-16-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-17-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-18-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-19-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-2-mult

/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:195: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=20, validation_data=([array([[..., verbose=1, callbacks=[<keras.ca..., initial_epoch=0, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


Epoch 1/20
5947/5947 [==============================] - 4591s 772ms/step - loss: 219.5699 - mean_squared_error: 0.1446 - shape_metric: 219.4253 - val_loss: 30.5096 - val_mean_squared_error: 0.0450 - val_shape_metric: 30.4647

Epoch 00001: val_loss improved from inf to 30.50965, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.1.hdf5
2019-06-03_11:30:11 - Epoch: 1, val_loss: 30.509645, val_mean_squared_error: 0.044953, val_shape_metric: 30.464692, loss: 219.832324, mean_squared_error: 0.144616, shape_metric: 219.687707
Epoch 2/20
5947/5947 [==============================] - 4462s 750ms/step - loss: 23.8928 - mean_squared_error: 0.0287 - shape_metric: 23.8640 - val_loss: 2517.5113 - val_mean_squared_error: 0.3113 - val_shape_metric: 2517.2000

Epoch 00002: val_loss did not improve from 30.50965
2019-06-03_12:44:33 - Epoch: 2, val_loss: 2517.511260, val_mean_squared_error: 0.311280, val_shape_metric: 2517.199982, loss: 23.870985, mean_squared_error: 0

In [ ]:
del dnn

In [ ]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [5,5,5,5]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.1
shape_constraint=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}'.format(nb_scales,layer_string,
                                                                               activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn= DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shape_constraint=shape_constraint)

In [ ]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'image-*-multihdu.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(write_path+'window/'+'Gaussian*')
win_files.sort()
print(win_files)


SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],
                        win_filename=win_files[2:],win_hdu=0,mom_hdu=1)

In [ ]:
del dnn

In [ ]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [5,5,5,5]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.01
shape_constraint=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}'.format(nb_scales,layer_string,
                                                                               activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn= DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shape_constraint=shape_constraint)

In [ ]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'image-*-multihdu.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(write_path+'window/'+'Gaussian*')
win_files.sort()
print(win_files)


SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],
                        win_filename=win_files[2:],win_hdu=0,mom_hdu=1)

In [13]:
from keras.models import Model, load_model
print(dnn.network_name)
dnn.network_name='ShapeNet2D_claire_sc3_layer3x3x3_relu_growthRate6_add_shape'
print(dnn.network_name)
print(dnn.gamma)
dnn.gamma = 0.1
print(dnn.gamma)
dnn.shape_loss
dnn.model=load_model('ShapeNet2D_claire_sc3_layer3x3x3_relu_growthRate6_pre_shape.hdf5', custom_objects={"shape_loss":dnn.shape_loss},compile=False)
initial_epoch=3

# dnn.train_generator(gal_files[2:4], gal_files[1], epochs=5, batch_size=32,
#                       model_file='', nb_img_per_file=1000, 
#                       validation_set_size=10000, noise_std=None, SNR=SNR, 
#                       noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
#                       psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
#                       deconv_mode=deconv_mode, win_validation_filename=win_files[1],
#                       win_filename=win_files[2:],win_hdu=0,mom_hdu=1,initial_epoch=initial_epoch)

ShapeNet2D_claire_sc3_layer3x3x3_relu_growthRate6_add_shape
ShapeNet2D_claire_sc3_layer3x3x3_relu_growthRate6_add_shape
0.1
0.1


In [16]:
from keras.optimizers import Adam
from keras import metrics
dnn.model.compile(optimizer = Adam(lr=1e-3), loss = dnn.shape_loss, metrics=[metrics.mse,dnn.shape_metric])

Tensor("loss_4/conv2d_48_loss/strided_slice_2:0", shape=(96, 96), dtype=float32) Tensor("loss_4/conv2d_48_loss/strided_slice:0", shape=(96, 96, 1), dtype=float32) Tensor("loss_4/conv2d_48_loss/strided_slice_1:0", shape=(96, 96, 1), dtype=float32) Tensor("loss_4/conv2d_48_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)


ValueError: Index out of range using input dim 3; input has only 3 dims for 'loss_4/conv2d_48_loss/strided_slice_3' (op: 'StridedSlice') with input shapes: [96,96,1], [4], [4], [4] and with computed input tensors: input[3] = <1 1 1 1>.

In [13]:
dnn.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_43 (Conv2D)              (None, 96, 96, 32)   288         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_151 (BatchN (None, 96, 96, 32)   128         conv2d_43[0][0]                  
__________________________________________________________________________________________________
activation_151 (Activation)     (None, 96, 96, 32)   0           batch_normalization_151[0][0]    
__________________________________________________________________________________________________
separable_

In [10]:
a

In [ ]:
del dnn

In [ ]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.01
shape_constraint=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}'.format(nb_scales,layer_string,
                                                                               activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn= DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shape_constraint=shape_constraint)

In [ ]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'image-*-multihdu.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(write_path+'window/'+'Gaussian*')
win_files.sort()
print(win_files)


SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:4], gal_files[1], epochs=3, batch_size=32,
                        model_file='', nb_img_per_file=1000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],
                        win_filename=win_files[2:],win_hdu=0,mom_hdu=1)

This example is to train a network for Tikhonov post processing.

If you want to train a denoiser on the ground truth images simply set:

    deconv_mode = None

    noiseless_img_hdu = target_hdu = 2

you do not need to provide the PSFs for this task.

Here some remarks about the memory usage if you work on GPUs

We have limited amount of VRAM on our GPUs (5GB for the Tesla K20c in the SAPPCN63 and 8GB on the Quadro P4000 in the DAPPCW149).

Inside the DeepNet class, there is a function computing the memory required for the model given its architecture and the size of one batch of data. This function is called automatically at the start of the training methods. It gives a rough idea of how much memory you will need to train the network. Keep in mind that it is only the memory needed for loading the parameters and on batch of data. All the intermediate computation will require as much if not more memory to do the training correctly.

If you use too much memory, either tensorflow will return a warning saying the memory is full but it can still optimize the training by itself to run, or it will return an OOM error and crash. In both case, the best is to try to optimize the memory allocation by either reducing the number of parameters of the DNN or the batch size.

Note that by default, when you start a Tensorflow session, it will try to pre-allocate 100% of the available GPU memory even when it doesn't need it all. Use the following bit of code to solve this issue:

In [ ]:
## extra imports to set GPU options
#import tensorflow as tf
#from keras import backend as k

###################################
# TensorFlow wizardry
#config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
#config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
# This line is optional, don't add it unless you really need to set a limit on the memory available for your process
# For instance, if you want to train 2 DNNs on the same GPU without one overlapping the memory needed by the other
# Change the value to set the percentage of memory allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.5 

# Create a session with the above options specified.
#k.tensorflow_backend.set_session(tf.Session(config=config))

# Now you can create/load your DNN

Same with multiple GPUs, Tensorflow will pre-allocate the whole memory of all the GPUs.
Use the following to prevent it (only when your station has several GPUs like the SAPPCN63):

In [ ]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0" #"0" for the 1st GPU or "1" to use the 2nd